In [7]:
import sys
from sys import platform
if platform  == 'win32':
    sys.argv=["_", r"P:\rmahajan14\capstone_data\data", r"P:\rmahajan14\capstone_data\cache"]
if platform == 'linux' or platform == 'linux2':
    def full_path(p):
        return os.path.abspath(os.path.expanduser(p))
    sys.argv=["_", r"/mnt/d/DiDi_Ride_Data/data", r"/mnt/d/DiDi_Ride_Data/Ride_Data_Cache"]

In [8]:
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from loader1 import *
from utils import *
from common import *
from experiment2 import *
# from sys import platform

import warnings
warnings.filterwarnings('ignore')

In [10]:
import time

Basic Coding 

In [16]:
df_final_no_decay, target_df_no_decay = get_final_df_reg(use_cache=True, decay='No Decay', spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
No Decay
../Ride_Data_Cache/active_times.msgpack exists
Decay Calculation done in 0.9689080715179443
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.21404314041137695
1f
Dumping to ../Ride_Data_Cache/hex_spatial_df.msgpack
spatial
Spatial Calculation done in 16705.16063094139


In [ ]:
df_final_old_decay, target_df_old_decay = get_final_df_reg(use_cache=False, decay='Old Decay')

In [18]:
# Using the function create_modified_active_time_through_decay2 
# instead of create_modified_active_time_through_decay
df_final_new_decay1, target_df_new_decay1 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=1, spatial_type=spatial_type)
df_final_new_decay10, target_df_new_decay10 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=10, spatial_type=spatial_type)
df_final_new_decay100, target_df_new_decay100 = get_final_df_reg(use_cache=True, decay='New Decay', mult_factor=100, spatial_type=spatial_type)

../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_1.msgpack exists
Decay Calculation done in 0.9310312271118164
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.20432305335998535
1f
../Ride_Data_Cache/hex_spatial_df.msgpack exists
spatial
Spatial Calculation done in 7.093277931213379
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_10.msgpack exists
Decay Calculation done in 0.9142119884490967
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features created in 0.23170924186706543
1f
../Ride_Data_Cache/hex_spatial_df.msgpack exists
spatial
Spatial Calculation done in 6.965825080871582
../Ride_Data_Cache/merged_orders.msgpack exists
orders
Decay Calculation
New Decay
../Ride_Data_Cache/idle_times_new_100.msgpack exists
Decay Calculation done in 0.9041521549224854
1e
../Ride_Data_Cache/features_orders.msgpack exists
Features 

### Modelling Code 

We have till now coded the following models 

1. Linear Regression 
2. Linear Regression with L1 Regularization
3. Linear Regression with L2 Regularization
4. Random Forest Regressor 

Now we split the data in a ratio of 75:25

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [23]:
def build_model(df, target_df, name, seed=0):
    # regression forest
    print(f'building regression model for {name}')
    X = df    
    xtrain, xtest, ytrain, ytest = train_test_split(X, target_df['target'], random_state=seed)

    sc = StandardScaler()

    xtrain_sc = sc.fit_transform(xtrain)

    rr = RandomForestRegressor(random_state=seed)
    rr.fit(xtrain_sc, ytrain)

    xtest_sc = sc.transform(xtest)
    
    print(rr.score(xtest_sc, ytest))
    
    return rr.score(xtest_sc, ytest)

In [25]:
df_list = [df_final_no_decay, df_final_old_decay, df_final_new_decay1, 
           df_final_new_decay10, df_final_new_decay100]
target_df_list = [target_df_no_decay, target_df_old_decay, target_df_new_decay1, 
                  target_df_new_decay10, target_df_new_decay100]
name_list = ['no_decay', 'old_decay', 'new_decay1', 'new_decay10', 'new_decay100']

In [26]:
result = {}
for df, target_df, name in zip(df_list, target_df_list, name_list):
    result[name] = build_model(df, target_df, name)

building regression model for no_decay
0.4192115651300975
building regression model for old_decay
0.6434035022904324
building regression model for new_decay1
0.608929696554577
building regression model for new_decay10
0.6906501087575441
building regression model for new_decay100
0.7523467893580604


In [27]:
result

{'no_decay': 0.4192115651300975,
 'old_decay': 0.6434035022904324,
 'new_decay1': 0.608929696554577,
 'new_decay10': 0.6906501087575441,
 'new_decay100': 0.7523467893580604}